In [2]:
import pandas as pd
import json
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Store dataset in a Pandas Dataframe
df = pd.read_csv('Kickstarter.csv')
df1 = pd.read_csv('Kickstarter001.csv')
df2 = pd.read_csv('Kickstarter002.csv')
df_ = pd.concat([df, df1, df2])
pd.set_option('display.max_columns', None)

In [ ]:
df.info()

In [ ]:
#effacer les variables non pertinentes pour notre analyse
df= df.drop(['converted_pledged_amount','country_displayable_name','creator','currency', 
                'currency_symbol','currency_trailing_code','current_currency','created_at','disable_communication',
                'friends','fx_rate','is_backing','is_starrable','is_starred','permissions','profile', 
                'slug','source_url','static_usd_rate','urls','pledged','photo','location','spotlight','id',
                'name','blurb','usd_exchange_rate'], axis=1)

#convertir les variables temporelles en dates et les diviser en colonnes weekday,
#month, year
df['deadline'] = pd.to_datetime(df['deadline'], unit='s')
df['deadline_date'] = df['deadline'].dt.date
df['deadline_weekday'] = df['deadline'].dt.weekday
df['deadline_month'] = df['deadline'].dt.month
df['deadline_year'] = df['deadline'].dt.year

df['launched_at'] = pd.to_datetime(df['launched_at'], unit='s')
df['launched_at_date'] = df['launched_at'].dt.date
df['launched_at_weekday'] = df['launched_at'].dt.weekday
df['launched_at_month'] = df['launched_at'].dt.month
df['launched_at_year'] = df['launched_at'].dt.year

df['state_changed_at'] = pd.to_datetime(df['state_changed_at'], unit='s')
df['state_changed_at_date'] = df['state_changed_at'].dt.date
df['state_changed_at_weekday'] = df['state_changed_at'].dt.weekday
df['state_changed_at_month'] = df['state_changed_at'].dt.month
df['state_changed_at_year'] = df['state_changed_at'].dt.year

#effacer les variables temporelles
df = df.drop(['deadline', 'launched_at', 'state_changed_at'], axis =1)

#ne garder que les modalités "successful" et "failed" de la variable cible 'state'
df = df[df['state'].isin(['failed', 'successful'])]

#ne garder que les valeurs de la clé "name" dans la colonne "category"
df['category'] = df['category'].apply(json.loads)
df['category'] = df['category'].apply(lambda x: x['name'])

#arrondir les montants
df['usd_pledged'] = df['usd_pledged'].round(2)

#Regroupement des données
##regrouper les catégories touchant au design dans la catégorie Design
df['category']=df['category'].replace(['Product Design','Graphic Design',
                                         'Architecture','Interactive Design',
                                         'Typography','Letterpress'], 'Design')

#regrouper les pays minoritaires dans des zones géographiques
df['country']=df['country'].replace(['DE','IT','DK','FR','LU','IE',
                                       'ES','PL','CH','NL','BE','SE',
                                       'NO', 'AT','GR','SI'], 'Europe')
df['country']=df['country'].replace(['JP','HK', 'SG'],'Asia')
df['country']=df['country'].replace(['AU','NZ'],'Oceania')

#Regrouper les dates avant 2013 dans 2013
df['state_changed_at_year']=df['state_changed_at_year'].replace([2009,2010,2011,2012], 2013)
df['deadline_year']=df['state_changed_at_year'].replace([2009,2010,2011,2012], 2013)
df['launched_at_year']=df['state_changed_at_year'].replace([2009,2010,2011,2012], 2013)

In [ ]:
#aperçu de notre jeu de données
df.head()

In [ ]:
#vérifier la présence de doublons dans le jeu de données
print("Il y a", df.duplicated().sum(), "doublons dans le jeu de données.")

#se débarrasser des doublons et créer un nouveau dataframe df_ qui servira pour la Visualisation
df_ = df.drop_duplicates(subset=None, keep="first").reset_index(drop=True)
print("Il reste", df_.duplicated().sum(), "doublons suite à l'opération de suppression. \n\n")

#vérifier la présence de NaN dans le jeu de données
print("Compte NaN:",df_.isna().sum().sum())
#Il n'y a pas de NaN.

### Explorer l'idée d'utiliser la durée des campagnes dans notre analyse

In [ ]:
#créer une nouvelle variabele "duree" pour afficher la durée réelle de la campagne
df_['duree'] = df_['deadline_date'] - df_['launched_at_date']
df_['duree_reelle'] = df_['state_changed_at_date'] - df_['launched_at_date']

#déterminer si des campagnes ont été prolongées
prolongation = df_[(df_['duree']) != (df_['duree_reelle'])]
#conclusion: il n'y a qu'un seul projet pour lequel la campagne a duré plus longtemps que prévu: id 595385287

#déterminer si des campagnes ont fini avant la deadline
prior = df_[(df_['deadline_date']) != (df_['state_changed_at_date'])]
#conclusion: non

#calculer la durée moyenne d'une campagne
df_['duree_reelle'].mean()

Conclusions de l'exploration:

Il n'y a qu'un seul projet pour lequel la campagne a duré plus longtemps que prévu: id 595385287

Il n'y a aucune campagne qui n'a été clôturée avant sa deadline.

La durée moyenne d'une campagne est de 32 jours.

Ainsi la durée d'une campagne ne semble pas pertinente pour expliquer sa réussite ou son échec.

In [ ]:
#suppression des dernières variables temporelles
df_ = df_.drop(['deadline_date','launched_at_date','state_changed_at_date','duree','duree_reelle' ], axis =1)

In [ ]:
##exporter df pour l'étape de la modélisation
df.to_csv('preprocessed_Modelisation.csv')

In [ ]:
#aperçu des données de df_ pour la visualisation
df_.head(10)

# Visualisation

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.countplot(y='category', data=df_,order = df_['category'].value_counts().index)
plt.title('Catégories de projet selon leur popularité auprès\n des contributeurs \n',fontsize=18)
plt.ylabel('Catégorie', rotation=360, fontsize=14, labelpad=60)
plt.xlabel('Nombre de contributeurs',fontsize=14, labelpad=20)
plt.show;


Sur le graphique "Catégories de projet selon leur popularité auprès des contributeurs" ci-dessus, on distingue assez clairement que 4 catégories sont plus populaires que les autres : Graphic Novels et Webcomics qui sont à peu près au même niveau puis Design et Cookbooks.

On peut ainsi supposer que les projets appartenant à ces 4 catégories ont plus de chances de réussir puisqu'ils convainquent davantage de contributeurs.

Pour évaluer la popularité d'une catégorie, on a utilisé le nombre de contributeurs total par catégorie de projet depuis 2009.

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y="category", data=df_[df_["state"]=="successful"],order = df_['category'].value_counts().index)
plt.title('Répartition des catégories pour les projets à succès \n',fontsize=18)
plt.ylabel('Catégorie', rotation=360, fontsize=14, labelpad=60)
plt.xlabel('Cumul de la catégorie',fontsize=14, labelpad=20)
plt.show;

Avec ce graphique, nous observons la répartition des catégories les plus à succés

la famille création artistique regroupant les catégories "Graphic Novels, Webcomics et Design" totalisent prés de 3000 reussites. Au contraire les "photo, Toys et Plays" enregistent les succés les plus bas avec un cumul d'environ 200 projets reussis.

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='state',data=df_, hue="staff_pick")
for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.2, p.get_height()), ha='center', va='top', color='white', size=18)
plt.title('Nombre de campagnes réussies et échouées selon leur\n présence dans la sélection des employés de Kickstarter \n\n',
          fontsize=18)
plt.xlabel('Statut des campagnes', labelpad=20, fontsize=14)
plt.ylabel('Nombre de campagnes', labelpad=20,fontsize=14)
;

Sur le graphique ci-haut, "Nombre de campagnes réussies et échouées selon leur présence dans la sélection des employés de Kickstarter" on remarque que plus de la moitié des campagnes réussies ont fait partie de la sélection des employés de Kickstarter et qu'une minorité des projets sélectionnés par les employés a échoué.

On peut en conclure qu'un projet faisant partie de la sélection des employés de la plateforme a plus de chances de réussir. Cela est probablement dû au fait qu'elle gagne en visibilité auprès des potentiels contributeurs en étant ajouté à leur liste "coup de coeur".

On pourrait également suggérer que les employés ont tendance à sélectionner des projets ayant initialement de grandes chances de réussir de par leurs connaissances et expériences. Ainsi on peut supposer qu'un projet inspire davantage confiance et envie lorsqu'il est se trouve dans cette liste.

Staff_pick = True distingue les projets qui ont été sélectionnés par les employés de Kickstarter de Staff_pick = False, qui indique le contraire.

In [ ]:
#identifier si le montant collecté dépend du nombre de contributeurs
plt.figure(figsize=(30,30))
sns.relplot(x='backers_count', y='usd_pledged', data=df_, alpha=0.7, height=8)
plt.ylabel('Montant collecté en usd', labelpad=20, fontsize=15)
plt.xlabel("Nombre de contributeurs", labelpad=20, fontsize=15)
plt.title('Relation entre le nombre de contributeurs \n et le montant total collecté \n\n',fontsize=18)
;

Le graphique qui précède permet de visualiser la relation entre le nombre de contributeurs et le montant collecté par campagne.

Nous pouvons remarquer 3 éléments:

le montant collecté total est relativement proportionnel au nombre de contributeurs
le nombre de contribueurs/campagne est majoritairement inférieur à 5000 pour un montant collecté total qui n'excède qu'assez rarement 500 000K.
les données sont donc assez rassemblées hormis quelques outliers qui se trouvent dans la partie haute du graphique. Ces outliers représentent potentiellement des campagnes qui ont très bien fonctionnées au niveau de la collecte de financement et/ou de la participation. Ceci dit il ne faut pas exclure la possibilité qu'il y ait des erreurs dans le jeu de données.

In [ ]:
#nb de campagnes ayant atteint leur objectif par mois
df_['state_changed_at_month'].astype(str)
df8 = df_[(df_["state"]=="successful") ]

plt.figure(figsize=(12,8))
sns.countplot(x=df8['state_changed_at_month'])

plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11], ['Janv', 'Fév', 'Mars', 'Avril',
                                          'Mai', 'Juin', 'Juil', 'Août', 'Sept',
                                            'Oct', 'Nov', 'Dec'])
plt.xlabel('Mois',fontsize=13, labelpad=20)
plt.ylabel('Nombre de campagnes réussies', fontsize=13, labelpad=20)
plt.title('Nombre de campagnes clôturées avec succès par mois\n\n',fontsize=15)
;

Le graphique ci-dessus illustre le nombre de campagnes reussies par mois

On y observe que les reussites sont au plus bas en début d'années. Cela pourrait s'expliquer par le manque de trésorerie aprés les fêtes de fin d'années. Les reussites décollent à partir de juillet, on passe de 200 reussites en Janvier à plus de 550 en juillet. Les niveaux max sont atteints en Octobre/Novembre, la période où l'on a tendance à investir dans des projets pour defiscaliser avant la fin d'année.

In [ ]:
plt.figure(figsize=(8, 8))
sns.countplot(x="state_changed_at_year",data=df_[df_["state"]=="successful"])
plt.title('Nombre de campagnes réussies au fil des années' ,fontsize=15)
plt.xlabel('Années', labelpad=20, fontsize=13)
plt.ylabel('Nombre de campagnes à succès', labelpad=20, fontsize=13)
plt.show;


Le graphique ci-haut montre l'évolution du nombre de campagnes reussies au fil des années.

On y observe à coup d'oeil une croissance exponentielle de l'activité en une dizaine d'années.

2009-2013 se caratcérise par un faible niveau d'activité, 2 explications à cela: phase de demarrage de l'entreprise et collecte des données moins regulières. Le bond intervient en 2014 avec une croissance de plus de 200 projets à succées par rapport à l'année précedante. Depuis la croissance se poursuit, jusqu'à atteidre un niveau record en 2022

In [ ]:
plt.figure(figsize=(8, 8))
sns.countplot(x=df_['usd_type'],data = df_['usd_pledged'])
plt.title('Montants collectés selon sa provenance' ,fontsize=15)
plt.xlabel('Provenance des collectes', labelpad=20, fontsize=13)
plt.ylabel('Montants des collectes', labelpad=20, fontsize=13)
plt.show;

Par notre graphique ci dessus nous voulons illustrer la répartition des montants collectés selon leur provenance et nous découvrons sans doute que la majorité des montants ont été collectés au niveau national avec près des 2/3 de la collecte. Cela nous montre que les contributeurs sont un peu frileux vis à vis des projets et à l'inverse ils connaissent un vrai engoument au niveau national.

In [ ]:
plt.figure(figsize=(8, 8))
sns.countplot(x="state_changed_at_weekday",data=df_[df_["state"]=="successful"])
plt.title('Nombre de campagnes réussies par jour de la semaine' ,fontsize=15)
plt.xlabel('jours de la semaine', labelpad=20, fontsize=13)
plt.ylabel('Nombre de campagnes réussies', labelpad=20, fontsize=13)
plt.show;

plt.xticks([0,1,2,3,4,5,6], ['Dimanche', 'lundi', 'Mardi', 'Mercredi',
                                          'Jeudi', 'Vendredi', 'Samedi']);

En créant la colonne ' state_change_at_weekday' nous avons voulu vérifier le nombre de campagnes lancée par jour de la semaine.Nous avons un faible démarrage en début de semaine par rapport à la moyenne des campagnes lancée et réussi. Nous constatons que c'est en milieu de semaine, les mercredis et Jeudis que notre compagne a plus de succès avec une forte augmentation les Mercredis ( success day).